## Transfer Learning

This notebook trains a visual recognition model using transfer learning that can classify between HotDogs and NotHotDogs.

The training data is taken from Kaggle [link](https://www.kaggle.com/datasets/dansbecker/hot-dog-not-hot-dog/)

#### Import Data (hot-dog-not-hot-dog) from Kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [ ]:
!kaggle datasets download -d dansbecker/hot-dog-not-hot-dog
!unzip \*.zip
!rm *.zip

In [ ]:
data_dir = "/content/"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

In [ ]:
import tensorflow as tf
import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#### First, train a simple CNN model.  As expected, the accuracy is quite low due to very few examples used for training.

#### Next, use transfer learning with Imagenet pre-trained model. After 15 epochs of training the model has around 95% of in-sample accuracy and 93%+ out-of-sample accuracy.

We use the train set to train a simple convolutional neural network (CNN).

In [ ]:
# Automatically create sets with labels
train_gen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

test_gen = ImageDataGenerator(rescale = 1.0/255.)

train_datagen = train_gen.flow_from_directory(train_dir, batch_size=20,
                                              class_mode='binary',
                                              target_size=(300, 300))
test_datagen =  test_gen.flow_from_directory(test_dir, batch_size=20,
                                             class_mode  = 'binary',
                                             target_size=(300, 300))

The `ImageDataGenerator` class is used to generate batches of tensor image data with real-time data augmentation. This means it can automatically create new variations of your images, which can improve your model if you don't have much data.

Here's what each parameter does:

- `rescale`: This is a value by which we will multiply the data before any other processing. Our original images consist of RGB coefficients in the 0-255 range, but these values would be too high for our models to process, so we target values between 0 and 1 instead by scaling with a 1/255 factor.

- `rotation_range`: This is a value in degrees (0-180), a range within which to randomly rotate pictures.

- `width_shift` and `height_shift`: These are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally.

- `shear_range`: Shear Intensity (Shear angle in counter-clockwise direction in degrees)

- `zoom_range`: This is a range for randomly zooming inside pictures.

- `horizontal_flip`: This is for randomly flipping half of the images horizontally -- relevant when there are no assumptions of horizontal asymmetry (e.g. real-world pictures).

- `fill_mode`: This is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.

Then, `flow_from_directory` is called on these `ImageDataGenerator` instances: `train_gen` and `test_gen`. This method loads images from the disk, applies rescaling, and resizes the images into the required dimensions.

- `train_dir` and `test_dir`: These are the directories where the training and testing data are located.

- `batch_size`: The size of the batches of data (default: 32).

- `class_mode`: Determines the type of label arrays that are returned. 'binary' means that the labels (two class problem) will be 1D binary labels.

- `target_size`: The dimensions to which all images found will be resized. In this case, it is (300, 300).

In [ ]:
# CNN with simple architecture
model = keras.Sequential([
        layers.Conv2D(32, (3,3), input_shape=(300,300,3), activation='relu'),
        layers.MaxPool2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPool2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPool2D(2,2),
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
            train_datagen,
            epochs=15,
            validation_data=test_datagen,
            verbose=2
            )

Examine accuracy and loss as a function of epochs.

In [ ]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_datagen, verbose=2)
print('Test accuracy:', test_acc)

# Transfer learning

Next, we use Imagenet pre-trained CNN model for transfer learning.

In [ ]:
# Import Imagenet pretrained model
base_model = keras.applications.Xception(
    weights='imagenet',
    input_shape=(300, 300, 3),
    include_top=False)

In [ ]:
# Freeze the model
base_model.trainable = False

In [ ]:
# Add layers

inputs = keras.Input(shape=(300, 300, 3))

# Building on Xception
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_datagen, epochs=15, validation_data=test_datagen)

In [ ]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_datagen, verbose=2)
print('Test accuracy:', test_acc)

You see the power of transfer learning!